# Comparing the performance of Python match pick library with rotational invariant method

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mrcfile
from scipy.spatial import cKDTree

In [ ]:
template = mrcfile.open("5MRC_10.00A_solvent-4.530V_real.mrc", permissive=True).data
grand_model = mrcfile.open("grandmodel.mrc", permissive=True).data


/workspaces/fle3d-experiment1/.venv/lib/python3.10/site-packages/mrcfile/mrcinterpreter.py:206: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)


In [ ]:
# gt = pd.read_csv(
#     "particle_locations.txt",
#     sep=r"\s+",
#     header=None,
#     names=["label", "z", "y", "x", "rot1", "rot2", "rot3"]
# )

# # proteins only
# gt = gt[~gt["label"].isin(["vesicle", "fiducial"])]

# # IMPORTANT: match coordinate order used in detection
# gt_xyz = gt[["z", "y", "x"]].values
# print("GT particles:", gt_xyz.shape)

GT particles: (1451, 3)


In [5]:
import pandas as pd

gt = pd.read_csv(
    "particle_locations.txt",
    sep=r"\s+",
    header=None,
    names=["label", "z", "y", "x", "rot1", "rot2", "rot3"]
)
print(gt["label"].unique())
# Keeping only protein particles (include vesicle + fiducial)
gt_protein = gt[~gt["label"].isin(["vesicle", "fiducial"])]

################### Limited to only 5MRC proteins
gt_protein = gt_protein[gt_protein["label"] == "5MRC"] 
############################


gt_xyz = gt_protein[["z", "y", "x"]].values

print("GT protein particles:", gt_xyz.shape)

['vesicle' 'fiducial' '1BXN' '1U6G' '3QM1' '1S3X' '4V94_fixed' '4CR2'
 '2CG9' '3H84' '3D2F' '1QVR' '5MRC' '3GL1' '3CF3']
GT protein particles: (131, 3)


In [6]:
def evaluate_detections(detected_xyz, gt_xyz, max_dist=8):
    tree = cKDTree(gt_xyz)
    matched = set()
    dists = []

    for p in detected_xyz:
        dist, idx = tree.query(p, k=1)
        if dist <= max_dist and idx not in matched:
            matched.add(idx)
            dists.append(dist)

    return np.array(dists), len(matched)